# Evaluating Nightshade

## Imports

In [1]:
import torch
from eval_nightshade import NightshadeEvaluator
from nightshade import Nightshade
from perturbation_methods import fgsm_penalty, pgd_penalty, nightshade_penalty
from data_process import get_dataset, get_poisoning_candidates, get_poisoned_dataset, create_mixed_dataset
from PIL import Image
import gc
import glob
import pickle
import numpy as np
import os

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Part 1

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
evaluator = NightshadeEvaluator(device=device)

methods = {
    'fgsm': fgsm_penalty,
    'pgd': pgd_penalty,
    'original': nightshade_penalty
}

results = {}
poisoned_imgs = {}
for name, method in methods.items():
    nightshade = Nightshade(target_concept=None, device=device, penalty_method=method)
    original_img = Image.open('test_images/dog.jpg')
    poisoned_img = nightshade.generate(original_img, target_concept='cat')
    poisoned_imgs[name] = poisoned_img

    results[name] = {
        'visual_metrics': evaluator.evaluate_perturbation(original_img, poisoned_img),
        'embedding_metrics': evaluator.evaluate_embedding_shift(original_img, poisoned_img, 'cat')
    }

    del nightshade
    gc.collect()
    torch.cuda.empty_cache()



In [ ]:
for name, result in results.items():
    print(f"Results for {name}:")
    print(f"Visual Metrics: {result['visual_metrics']}")
    print(f"Embedding Metrics: {result['embedding_metrics']}")
    print()
    # Display the poisoned image
    display(poisoned_imgs[name])

## Part 2

In [ ]:
# get a list of poisoning candidates
clean_dataset = get_dataset('annotations/captions_train2014.json', 'train2014', 5000)
poisoning_candidates = get_poisoning_candidates(clean_dataset, 'dog', 100)

In [ ]:
candidate_files = glob.glob("poisoning_candidates/dog_*.p")
candidates = [pickle.load(open(f, 'rb')) for f in candidate_files]

for name, method in methods.items():
    output_dir = f'poisoned_images/{name}'
    nightshade = Nightshade(target_concept=None, device=device, penalty_method=method)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for candidate in candidates:
        
        original_img = Image.fromarray(candidate['img'])
        poisoned_img = nightshade.generate(original_img, target_concept='cat')
        poisoned_data = {
            'img': np.array(poisoned_img),
            'text': candidate['text']
        }
        pickle.dump(poisoned_data, open(f"{output_dir}/{name}_{candidate['image_id']}.p", 'wb'))
    print(f'Saved {len(candidates)} poisoned images for {name} method in {output_dir}')
    del nightshade
    gc.collect()
    torch.cuda.empty_cache()


### Train Stable Diffusion on each sample

In [7]:
num_poisoned = [10, 25, 50, 100]
methods = {
    'fgsm': fgsm_penalty,
    'pgd': pgd_penalty,
    'original': nightshade_penalty
}

clean_df = get_dataset('annotations/captions_train2014.json', 'train2014', 10000)
poisoned_datasets = {}
for name, method in methods.items():
    for num in num_poisoned:
        poisoned_dataset = get_poisoned_dataset(f'poisoned_images/{name}', limit=num)
        poisoned_datasets[f"{name}_{num}"] = poisoned_dataset


mixed_dataset = create_mixed_dataset(clean_df, poisoned_datasets['original_50'])

Processing unique images:   0%|          | 0/82783 [00:00<?, ?it/s]

Loaded 10000 unique image entries
Loaded 10 poisoned entries from poisoned_images/fgsm
Loaded 25 poisoned entries from poisoned_images/fgsm
Loaded 50 poisoned entries from poisoned_images/fgsm
Loaded 100 poisoned entries from poisoned_images/fgsm
Loaded 10 poisoned entries from poisoned_images/pgd
Loaded 25 poisoned entries from poisoned_images/pgd
Loaded 50 poisoned entries from poisoned_images/pgd
Loaded 100 poisoned entries from poisoned_images/pgd
Loaded 10 poisoned entries from poisoned_images/original
Loaded 25 poisoned entries from poisoned_images/original
Loaded 50 poisoned entries from poisoned_images/original
Loaded 100 poisoned entries from poisoned_images/original


Mixing datasets:   0%|          | 0/50 [00:00<?, ?it/s]

Created mixed dataset with 50/10000 poisoned entries.	 0.50% poisoned.


In [ ]:
from train_sd import train_model

train_model(mixed_dataset, 'output_models/', 10, 1)

Epoch 1/2:   0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 2/2:   0%|          | 0/10000 [00:00<?, ?it/s]

Training complete. Model saved to output_models/


In [6]:
from diffusers import StableDiffusionPipeline
from train_sd import load_lora_into_pipeline

pipe = StableDiffusionPipeline.from_pretrained(
    'runwayml/stable-diffusion-v1-5', 
    torch_dtype=torch.float16, 
    safety_checker=None
).to('cuda')

pipe = load_lora_into_pipeline(pipe, 'output_models/lora_adapter')

prompt = 'dog'
image = pipe(prompt).images[0]
image.save("sample.png")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/50 [00:00<?, ?it/s]